<a href="https://colab.research.google.com/github/BuczynskiRafal/sentiment_snalysis/blob/main/sentiment_snalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import bibliotek


In [10]:
import numpy as np
import pandas as pd
import plotly.express as px
import sklearn

np.random.seed(42)
np.set_printoptions(precision=6, suppress=True, edgeitems=10, linewidth=1000, formatter=dict(float=lambda x: f'{x:.2f}'))
sklearn.__version__

'1.0.2'

# Wygenerowanie danych

In [11]:
documents = [
    'Today is Friday',
    'I like Friday',
    'Today I am going to learn Python.',
    'Friday, Friday!!!'
]

print(documents)

['Today is Friday', 'I like Friday', 'Today I am going to learn Python.', 'Friday, Friday!!!']


# Wektoryzacja tekstu

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(documents)

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [14]:
vectorizer.fit_transform(documents).toarray()

array([[0, 1, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 1, 1],
       [0, 2, 0, 0, 0, 0, 0, 0, 0]])

In [16]:
vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['am', 'friday', 'going', 'is', 'learn', 'like', 'python', 'to', 'today']

In [19]:
df = pd.DataFrame(data=vectorizer.fit_transform(documents).toarray(),
                  columns=vectorizer.get_feature_names())
df

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,am,friday,going,is,learn,like,python,to,today
0,0,1,0,1,0,0,0,0,1
1,0,1,0,0,0,1,0,0,0
2,1,0,1,0,1,0,1,1,1
3,0,2,0,0,0,0,0,0,0


In [21]:
vectorizer.vocabulary_

{'am': 0,
 'friday': 1,
 'going': 2,
 'is': 3,
 'learn': 4,
 'like': 5,
 'python': 6,
 'to': 7,
 'today': 8}

In [22]:
# powyżej pokzane kodowanie
# chcąc zakodować inny tekst można użyć metody transform()
# trzeba jednak zaznaczyć że model jest dopasowany do innych danych i nazwet nie zakoduje słoa którego nie widział
vectorizer.transform(['Friday morning']).toarray()


array([[0, 1, 0, 0, 0, 0, 0, 0, 0]])

# Wektoryzacja tekstu - bigramy - sens zdania

In [23]:
bigram = CountVectorizer(ngram_range=(1, 2), min_df=1)
bigram.fit_transform(documents).toarray()

array([[0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [24]:
bigram.vocabulary_

{'am': 0,
 'am going': 1,
 'friday': 2,
 'friday friday': 3,
 'going': 4,
 'going to': 5,
 'is': 6,
 'is friday': 7,
 'learn': 8,
 'learn python': 9,
 'like': 10,
 'like friday': 11,
 'python': 12,
 'to': 13,
 'to learn': 14,
 'today': 15,
 'today am': 16,
 'today is': 17}

In [27]:
df = pd.DataFrame(data=bigram.fit_transform(documents).toarray(),
                  columns=bigram.vocabulary_)
df

,today,is,friday,today is,is friday,like,like friday,am,going,to,learn,python,today am,am going,going to,to learn,learn python,friday friday
0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1
1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
2,1,1,0,0,1,1,0,0,1,1,0,0,1,1,1,1,1,0
3,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# TFIDF Transformer - metoda obliczania ważności słów w dokumencie oparta o częstość ich wystąpień

In [28]:
documents = [
    'Friday morning',
    'Friday chill',
    'Friday - morning',
    'Friday, Friday morning!!!'
]

print(documents)

['Friday morning', 'Friday chill', 'Friday - morning', 'Friday, Friday morning!!!']
